# __Scrape Historical Fight Data__

### Import Libraries

In [1]:
import requests
import pandas as pd
import string
import IPython
import time
from bs4 import BeautifulSoup
from pandas.plotting import scatter_matrix
import os 
import re
from dateutil import parser
import datetime
import random

%matplotlib inline

/anaconda3/envs/fight_prediction/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/envs/fight_prediction/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Get links for each UFC event

In [2]:
all_events_url = "http://ufcstats.com/statistics/events/completed?page=all"
all_events_page = requests.get(all_events_url)
all_events_soup = BeautifulSoup(all_events_page.content, 'html.parser')

In [3]:
table = all_events_soup.find('table')
events = table.findAll('a')
# Pop the first link because it is for the next UFC event
events.pop(0,)

<a class="b-link b-link_style_white" href="http://ufcstats.com/event-details/a7a79b8efbceaaac">
                          UFC Fight Night: Ngannou vs. Velasquez
                        </a>

### Create Dictionary for Statistics

In [4]:
%%time
d = {}
# Make a for loop with dictionary to get a bunch of these thigngs. 
# Create empty lists for fighters and winner
fighters_1 = []
fighters_2 = []
end_rounds = []
end_times = []
winners = []

# General information
dates = []
fight_urls = []

# Debugging
event_error_urls = []
fight_error_urls = []

num_rounds = 5
cur_round = 1
while(cur_round <= num_rounds):
    fighter_index = 1
    while fighter_index <= 2:
        # General Fight Stats
        d["f{0}_r{1}_kds".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_sigstrikes_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_sigstrikes_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_sigstrikes_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_strikes_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_strikes_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_tds_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_tds_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_subs_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_passes".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_revs".format(fighter_index, cur_round)] = []
        
        # Stats Regarding Signficant Strikes Target and Distance
        d["f{0}_r{1}_ss_head_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_head_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_body_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_body_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_leg_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_leg_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_dist_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_dist_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_clinch_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_clinch_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_ground_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_ss_ground_a".format(fighter_index, cur_round)] = []
        fighter_index += 1
    cur_round += 1

CPU times: user 147 µs, sys: 1e+03 ns, total: 148 µs
Wall time: 151 µs


In [5]:
%%time
page_count = 1
num_errors = 0
date_cutoff = datetime.datetime(2010,1,1,0,0)

for event in events:
    try:
        # Setup  url, page, and soup for EVENT
        event_url = event['href']
        time.sleep(random.uniform(0,.5))
        event_page = requests.get(event_url)
        event_soup = BeautifulSoup(event_page.content, 'html.parser')
        
        # Extract date
        date_text = event_soup.find("li",{"class":"b-list__box-list-item"}).text.strip().strip("Date:").strip("\n").strip()
        date = parser.parse(date_text)
        
        if date < date_cutoff:
            break

        # Parse table for statistics
        table = event_soup.find('table')
        rows = table.findAll('tr',onclick=True)
        for row in rows:
            # Setup FIGHT url, page, and soup for FIGHT
            fight_url = row['onclick'][7:-2]
            time.sleep(random.uniform(0,.5))
            
            try:
                fight_page = requests.get(fight_url)
                fight_soup = BeautifulSoup(fight_page.content, 'html.parser')

                # Add fighters
                section = fight_soup.findAll("div",{"class":"b-fight-details__person"})

                winner = ''
                for i,fighter in enumerate(section,1):
                    if i == 1:
                        fighter_1 = ' '.join(fighter.text.replace('\n','').strip().split()[1:])

                    elif i == 2:
                        fighter_2 = ' '.join(fighter.text.replace('\n','').strip().split()[1:])

                    # Add winner
                    if(fighter.text.strip()[0] == "W"):
                        winner = ' '.join(section[0].text.replace('\n','').strip().split()[1:])

                if winner == '':
                    winner = 'No Winner'
                    
                # Add End Round and Time
                fight_details = fight_soup.find("p",{"class":"b-fight-details__text"})
                fight_details_i = fight_details.findAll("i")
                end_rounds.append(fight_details_i[3].text.split()[1]) # end round
                end_times.append(fight_details_i[5].text.replace("\n","").strip().split()[1]) # end time

                # SCRAPE ROUND SPECIFIC STATS
                stats_tables = fight_soup.findAll("table",{"class","b-fight-details__table js-fight-table"})
                round_stats_table = stats_tables[0]
                p_r_stats = round_stats_table.findAll('p',{'class':"b-fight-details__table-text"})

                # Create Indices
                stat_ind = 2
                round_ind = 1

                while(round_ind <= 5):
                    fighter_ind = 1
                    while fighter_ind <= 2:
                        # Try except is used because many of the fights don't go full the distance
                        try: 
                            # Scrape Knockdowns
                            d['f{0}_r{1}_kds'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip())
                            stat_ind += 2
                            # Scrape Significant Strikes
                            d['f{0}_r{1}_sigstrikes_l'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[0]) 
                            d['f{0}_r{1}_sigstrikes_a'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[2]) 
                            # Scrape Strikes
                            stat_ind += 4
                            d['f{0}_r{1}_strikes_l'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[0]) 
                            d['f{0}_r{1}_strikes_a'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[2])
                            # Scrape Takedowns
                            stat_ind += 2
                            d['f{0}_r{1}_tds_l'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[0]) 
                            d['f{0}_r{1}_tds_a'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[2])
                            # Scrape Subs
                            stat_ind += 4
                            d['f{0}_r{1}_subs_a'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip())
                            # Scrape Pass
                            stat_ind += 2
                            d['f{0}_r{1}_passes'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip())  
                            # Scrape Rev.
                            stat_ind += 2
                            d['f{0}_r{1}_revs'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip())
                        except:
                            d['f{0}_r{1}_kds'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_sigstrikes_l'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_sigstrikes_a'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_strikes_l'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_strikes_a'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_tds_l'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_tds_a'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_subs_a'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_passes'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_revs'.format(fighter_ind, round_ind)].append(None)  


                        if fighter_ind == 1:             
                            stat_ind = 3+20*(round_ind-1)
                        elif fighter_ind == 2: 
                            stat_ind = 2+20*(round_ind -1)

                        fighter_ind +=1

                    round_ind += 1

                # SCRAPE STYLE SPECIFIC STATS
                round_ss_stats_table = stats_tables[1]
                p_r_ss_stats = round_ss_stats_table.findAll('p',{'class':"b-fight-details__table-text"})

                # Create Indices
                stat_ind = 6
                round_ind = 1

                while(round_ind <= 5):
                    fighter_ind = 1
                    while fighter_ind <= 2:
                        # Try except is used because many of the fights don't go full the distance
                        try: 
                            # Scrape SigStrikes to Head
                            d['f{0}_r{1}_ss_head_l'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[0])
                            d['f{0}_r{1}_ss_head_a'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[2])    
                            # Scrape SigStrikes to Body
                            stat_ind +=2
                            d['f{0}_r{1}_ss_body_l'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[0])
                            d['f{0}_r{1}_ss_body_a'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[2])
                            # Scrape SigStrikes to Leg
                            stat_ind +=2
                            d['f{0}_r{1}_ss_leg_l'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[0])
                            d['f{0}_r{1}_ss_leg_a'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[2])
                            # Scrape SigStrikes from Dist
                            stat_ind +=2
                            d['f{0}_r{1}_ss_dist_l'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[0])
                            d['f{0}_r{1}_ss_dist_a'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[2])
                            # Scrape SigStrikes from Clinch
                            stat_ind +=2
                            d['f{0}_r{1}_ss_clinch_l'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[0])
                            d['f{0}_r{1}_ss_clinch_a'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[2])
                            # Scrape SigStrikes from Ground
                            stat_ind +=2
                            d['f{0}_r{1}_ss_ground_l'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[0])
                            d['f{0}_r{1}_ss_ground_a'.format(fighter_ind, round_ind)].append(p_r_ss_stats[stat_ind].text.replace('\n','').strip().split()[2])

                        except:
                            d['f{0}_r{1}_ss_head_l'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_head_a'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_body_l'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_body_a'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_leg_l'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_leg_a'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_dist_l'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_dist_a'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_clinch_l'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_clinch_a'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_ground_l'.format(fighter_ind, round_ind)].append(None)
                            d['f{0}_r{1}_ss_ground_a'.format(fighter_ind, round_ind)].append(None)  


                        if fighter_ind == 1:             
                            stat_ind = 6+18*(round_ind-1)
                        elif fighter_ind == 2: 
                            stat_ind = 7+18*(round_ind -1)

                        fighter_ind +=1

                    round_ind += 1

                # Append data to lists
                fighters_1.append(fighter_1)
                fighters_2.append(fighter_2)
                winners.append(winner)
                fight_urls.append(fight_url)
                dates.append(date)
            except:
                fight_error_urls.append(fight_url)
    except:
        num_errors += 1
        print("There are " + str(num_errors))

        
    page_count +=1
    
#    if page_count == 3:
#        break

CPU times: user 3min 20s, sys: 3.99 s, total: 3min 24s
Wall time: 3h 9min 28s


In [6]:
round_stats_df = pd.DataFrame(d)

gen_stats_dict = {'fighter_1':fighters_1,
                  'fighter_2':fighters_2,
                  'end_round': end_rounds,
                  'end_time': end_times,
                  'winner':winners,           
                  'url': fight_urls,
                  'date': dates}

gen_stats_df = pd.DataFrame(gen_stats_dict, columns=gen_stats_dict.keys())

all_stats_df = pd.concat([gen_stats_df, round_stats_df], axis=1)
all_stats_df

,fighter_1,fighter_2,end_round,end_time,winner,url,date,f1_r1_kds,f1_r1_sigstrikes_l,f1_r1_sigstrikes_a,...,f2_r5_ss_body_l,f2_r5_ss_body_a,f2_r5_ss_leg_l,f2_r5_ss_leg_a,f2_r5_ss_dist_l,f2_r5_ss_dist_a,f2_r5_ss_clinch_l,f2_r5_ss_clinch_a,f2_r5_ss_ground_l,f2_r5_ss_ground_a
0,"Israel Adesanya ""The Last Stylebender""","Anderson Silva ""The Spider""",3,5:00,"Israel Adesanya ""The Last Stylebender""",http://ufcstats.com/fight-details/4eb91cfc892a...,2019-02-09,0,20,44,...,None,None,None,None,None,None,None,None,None,None
1,"Lando Vannata ""Groovy""","Marcos Mariano ""Dhalsim""",1,4:55,"Lando Vannata ""Groovy""",http://ufcstats.com/fight-details/aa71cf96ba4e...,2019-02-09,0,20,40,...,None,None,None,None,None,None,None,None,None,None
2,Rani Yahya,Ricky Simon,3,5:00,Rani Yahya,http://ufcstats.com/fight-details/069e3e704cbf...,2019-02-09,0,21,62,...,None,None,None,None,None,None,None,None,None,None
3,Montana De La Rosa,"Nadia Kassem ""187""",2,2:37,Montana De La Rosa,http://ufcstats.com/fight-details/fd1854062406...,2019-02-09,0,10,16,...,None,None,None,None,None,None,None,None,None,None
4,"Jimmy Crute ""The Brute""","Sam Alvey ""Smile'N""",1,2:49,"Jimmy Crute ""The Brute""",http://ufcstats.com/fight-details/e4862c4f2011...,2019-02-09,1,14,32,...,None,None,None,None,None,None,None,None,None,None
5,"Devonte Smith ""King Kage""","Dong Hyun Ma ""Maestro""",1,3:53,"Devonte Smith ""King Kage""",http://ufcstats.com/fight-details/5ff8463edf81...,2019-02-09,1,19,43,...,None,None,None,None,None,None,None,None,None,None
6,"Shane Young ""Sugar""","Austin Arnett ""Golden Boy""",3,5:00,"Shane Young ""Sugar""",http://ufcstats.com/fight-details/49f7186e1e3c...,2019-02-09,0,39,108,...,None,None,None,None,None,None,None,None,None,None
7,"Kai Kara-France ""Don't Blink""",Raulian Paiva,3,5:00,"Kai Kara-France ""Don't Blink""",http://ufcstats.com/fight-details/51db35a3cf22...,2019-02-09,0,12,39,...,None,None,None,None,None,None,None,None,None,None
8,"Teruto Ishihara ""Yashabo""","Kyung Ho Kang ""Mr. Perfect""",1,3:59,"Teruto Ishihara ""Yashabo""",http://ufcstats.com/fight-details/b247a1900cfa...,2019-02-09,1,17,47,...,None,None,None,None,None,None,None,None,None,None
9,"Callan Potter ""The Rockstar""","Jalin Turner ""The Tarantula""",1,0:53,"Callan Potter ""The Rockstar""",http://ufcstats.com/fight-details/11c0ef974973...,2019-02-09,0,3,4,...,None,None,None,None,None,None,None,None,None,None


In [8]:
# save as csv
all_stats_df.to_csv("../data/full_stats_v2.csv")

## Testing for Event Page

In [3]:
te_url = 'http://ufcstats.com/event-details/84283233ec42be5f'
te_page = requests.get(te_url)
te_soup = BeautifulSoup(te_page.content, 'html.parser')

text = te_soup.find("li",{"class":"b-list__box-list-item"}).text.strip().strip("Date:").strip("\n").strip()

In [78]:
text

'February 02, 2019'

In [79]:
parser.parse(text)

datetime.datetime(2019, 2, 2, 0, 0)

### Testing for Fight Page

In [14]:
tf_url = 'http://ufcstats.com/fight-details/904e0e46392dff93'
tf_page = requests.get(tf_url)
tf_soup = BeautifulSoup(tf_page.content,'html.parser')

In [15]:
fs_table = tf_soup.findAll("table",{"class","b-fight-details__table js-fight-table"}) ## Start here
table_0 = fs_table[0]

In [16]:
p_ele = table_0.findAll('p',{'class':"b-fight-details__table-text"})

In [17]:
print(p_ele[14].text.replace('\n','').strip()) # Sub attempt f1
print(p_ele[16].text.replace('\n','').strip()) # Pass attempt f1
print(p_ele[18].text.replace('\n','').strip()) # Reversal attempt f1

0
2
0


In [7]:
t_url = "http://ufcstats.com/fight-details/4e8b2357fea211e9"
t_page = requests.get(t_url)
t_soup = BeautifulSoup(t_page.content, 'html.parser')

fight_details = t_soup.find("p",{"class":"b-fight-details__text"})
fight_details_i = fight_details.findAll("i")

In [23]:
fight_details_i[3].text.split()[1] # round

'1'

In [21]:
fight_details_i[5].text.replace("\n","").strip().split()[1] # time

'3:17'